In [1]:
#Get Training Files
import os
import pandas as pd
in_data_array={}
out_data_array={}
for file in os.listdir('/kaggle/input/nfl-big-data-bowl-2026-prediction/train'):
    if file[0:2]=='in':
        week=int(file[-6:-4])
        dataFrame=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+file)
        in_data_array[week]=dataFrame
    elif file[0:2]=='ou':
        week=int(file[-6:-4])
        dataFrame=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+file)
        out_data_array[week]=dataFrame

In [2]:
#Acquire Actual Data
actual_data=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv')
print(actual_data)

         game_id  play_id  nfl_id  frame_id
0     2024120805       74   54586         1
1     2024120805       74   54586         2
2     2024120805       74   54586         3
3     2024120805       74   54586         4
4     2024120805       74   54586         5
...          ...      ...     ...       ...
5832  2025010515     3902   55112        26
5833  2025010515     3902   55112        27
5834  2025010515     3902   55112        28
5835  2025010515     3902   55112        29
5836  2025010515     3902   55112        30

[5837 rows x 4 columns]


In [3]:
#Write to output file
with open('/kaggle/working/submission.csv','w+') as file:
    file.write("id,x,y")
